### Plasma Sheath, Cylindrical Langmuir Probe, Radial model

First, we set the kind of plasma. Uncomment the line that corresponds

In [1]:
particle_masses_select = "Argon and electrons"
# particle_masses_select == "Neon and electrons"
# particle_masses_select == "Oxigen O+ and O-"
# particle_masses_select = "Enter masses"

Now, some imports and an initializer

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import functions as f
import copy

def none_initializer():
    for i in range(100):
        yield None

Definition of psysical constants that we will need

In [3]:
class Constants:
    kb, k, m_e, q_e, uma, *_ = none_initializer()

constants = Constants()

constants.kb = 1.380648e-23     #Boltzmann constant, SI
constants.k = 8.61732739e-5     #k_B/e, SI
constants.m_e = 5.4858e-4       #electron mass, umas
constants.q_e = 1.602176e-19    #electron absolute charge, SI
constants.uma = 1.66054e-27     #SI

Creation of the Plasma object in Python, with an assistance integer getter, which accepts "m_e" as the mass of the electron for a bimaxwellian plasma

In [4]:
def get_int(s):
    while True:
        try:
            x = int(input(s))
            break
        except ValueError:
            if x == "m_e":
                x = constants.m_e
            else:
                print("No valid number.  Try again...")
    return x

class Plasma:
    A, B, *_ = none_initializer()

plasma = Plasma()

if particle_masses_select == "Argon and electrons":
    plasma.A = 39.948        #Argon cation mass, umas
    plasma.B = constants.m_e           #defined above, electron mass, umas
    
elif particle_masses_select == "Neon and electrons":
    plasma.A = 20.18         #Neon cation mass, umas
    plasma.B = constants.m_e           #defined above, electron mass, umas
    
elif particle_masses_select == "Oxigen O+ and O-":
    plasma.A = 15.9989       #Oxygen single charged cation mass, umas
    plasma.B = 15.9999       #Oxygen single charged anion mass, umas
else:
    plasma.A = get_int("Enter positive ion mass, in umas")
    plasma.B = get_int("Enter negative ion mass, in umas")
print("Positive ion mass = {0} and Negative ion mass = {1}".format(plasma.A, plasma.B))

Positive ion mass = 39.948 and Negative ion mass = 0.00054858


Creaion of global variables: Ion current, positive ion to electron temperature ratio...

In [5]:
class Globals:
    beta, gamma, alpha0, kappa, xp, Ip, I01, I02, yp, yf, sol_Euler, xl, x0a, xMAX, *_ = none_initializer()

glob = Globals()

glob.beta = 0.1            #positive ion temperature to electron temperature ratio
glob.gamma = 10.0          #electron temperature to negative ion temperature ratio
glob.alpha0 = 0.5          #negative ion density to electron density ratio
glob.kappa = 2.0           #thermodynamic adiabatic index
#glob.vmas0 = 0.5           #Bohm velocity
glob.xp = 1.0              #probe radius to Debye length ratio

glob.Ip = 5.0              #adimensional ion current
glob.I01 = None            #negative ion current constant
glob.I02 = None            #electron current constant
glob.yp = None             #adimensional probe potential
glob.yf = None             #adimensional floating potential

glob.sol_Euler = 1         #index to select Euler's equation branch for the solution

glob.xl = None             #maximum x for singularity, according to theory
glob.x0a = None            #singularity x for bounded quasineutral solution
glob.xMAX = 50.0           #Runge-Kutta integration limit if singularity x is small

Configuration variables

In [6]:
class Configuration:
    cylindrical, numprec, graph, save, warn, time_calculation_reduction, *_ = none_initializer()

configuration = Configuration()

configuration.cylindrical = 1
configuration.numprec = 1e-9
configuration.graph = 1
configuration.save = 0
configuration.warn = 1
configuration.time_calculation_reduction = 3     #reduces compute time, 3 is ok
configuration.xinf = 20
#Ap = np.pi*6e-3*2e-4 ??

Global object that stores all the previous configuration. It is passed to all functions, given that python has no true global variables.

In [7]:
class BetaGammaSheath:
    c = copy.deepcopy(constants)
    p = copy.deepcopy(plasma)
    g = copy.deepcopy(glob)
    config = copy.deepcopy(configuration)
    
bg = BetaGammaSheath()

Calculation of the sheath

In [8]:
import sheath
[x_array, y_array, z_array, N_array] = sheath.cylindrical(bg)

Nrk = 83.33333333333333; Ip = 5.0; beta = 0.1
bg_sheath looking for singuarity loop, xu - xd = 9.682458365518544; xtest = 9.682458365518544
bg_sheath looking for singuarity loop, xu - xd = 4.841229182759273; xtest = 14.523687548277817
bg_sheath looking for singuarity loop, xu - xd = 2.4206145913796355; xtest = 12.103072956898181
bg_sheath looking for singuarity loop, xu - xd = 1.2103072956898178; xtest = 13.313380252587999
bg_sheath looking for singuarity loop, xu - xd = 0.6051536478449098; xtest = 13.918533900432909
bg_sheath looking for singuarity loop, xu - xd = 0.302576823922454; xtest = 13.615957076510455
bg_sheath looking for singuarity loop, xu - xd = 0.151288411961227; xtest = 13.767245488471682
bg_sheath looking for singuarity loop, xu - xd = 0.07564420598061439; xtest = 13.842889694452296
bg_sheath looking for singuarity loop, xu - xd = 0.037822102990306306; xtest = 13.805067591461988
bg_sheath looking for singuarity loop, xu - xd = 0.01891105149515404; xtest = 13.78615653996

Plot the potential in k_bT_e/e units versus the distance to the axis in lambda_D units

In [12]:
%matplotlib notebook
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(x_array, y_array)


<IPython.core.display.Javascript object>

In [10]:
print("Positive ion mass = {0} and Negative ion mass = {1}".format(bg.p.A, bg.p.B))

Positive ion mass = 39.948 and Negative ion mass = 0.00054858
